### Riddler Classic

Congratulations, you’ve made it to the fifth round of The Squiddler — a competition that takes place on a remote island. In this round, you are one of the 16 remaining competitors who must cross a bridge made up of 18 pairs of separated glass squares. Here is what the bridge looks like from above:
    
To cross the bridge, you must jump from one pair of squares to the next. However, you must choose one of the two squares in a pair to land on. Within each pair, one square is made of tempered glass, while the other is made of normal glass. If you jump onto tempered glass, all is well, and you can continue on to the next pair of squares. But if you jump onto normal glass, it will break, and you will be eliminated from the competition.

You and your competitors have no knowledge of which square within each pair is made of tempered glass. The only way to figure it out is to take a leap of faith and jump onto a square. Once a pair is revealed — either when someone lands on a tempered square or a normal square — all remaining competitors take notice and will choose the tempered glass when they arrive at that pair.

On average, how many of the 16 competitors will survive and make it to the next round of the competition?


### Sim Approach: Assumptions
- We have participants guess at each step. 
- Every pair has two options, where 0 or 1 represents the tempered glass and must be matched. 


### A Note on Solution:

It was absolutely unecessary for me to have a `dataclass` here, but tryint to play around with some stuff I am learning in `Fluent Python` (which makes it clear to not overuse classes, but hey here we are). 

The nice thing about using a dataclass is I could shuffle contestants and determine which survive each round (attempting to justify the decision). 

There are a lot of slow decisions in the first iteration, this will be cleaned. 

In [3]:
from dataclasses import dataclass
import random 

class Bridge():
    def __init__(self, size = 16):
        self.size = size
        
    def mapTempered(self):
        """Function to randomly determine if tempered is 0 or 1"""
        self.bridge = random.choices([0,1], k=self.size)
        
    def traversal(self, contestants):
        """
        Assuming no shenanigans, it should only take 1 contestant at most to traverse a single step.
        If a contestant falls through, the next contests should avoid the window they fell through
        """
        i = 0 # initial contestant
        for step in self.bridge:
            if contestants[i].choose() != step:
                contestants[i].alive = 0
                i += 1
            else:
                continue
        return contestants
    
@dataclass
class Contestant:
    """class representating a competitor in Squid games"""
    number: int
    alive: int = 1 # they all start out alive

    def choose(self):
        return random.choices([0,1], k=1)[0]

### Run Some Sims: 

We could surely solve this faster in numpy, that is next. 

In [4]:
import time 
sims = [1_000, 10_000, 100_000, 1_000_000, 2_000_000]
for sim in sims:
    start = time.time()
    tot_alive = 0
    for _ in range(sim):
        sequence = [Contestant(i+1) for i in range(16)] # our new sequence -> really unecessary (SLOW)
        b = Bridge(16)
        b.mapTempered()
        sequence = [Contestant(i+1) for i in range(16)]
        output = b.traversal(sequence)
        tot_alive += sum([c.alive for c in output])
    end = time.time()
    print(f"Average survivors for {sim} sims: {tot_alive / sim}")
    print(f"Total time: {end - start:.2f}\n")

Average survivors for 1000 sims: 7.876
Total time: 0.05

Average survivors for 10000 sims: 7.9949
Total time: 0.20

Average survivors for 100000 sims: 7.99934
Total time: 1.88

Average survivors for 1000000 sims: 7.998777
Total time: 18.81

Average survivors for 2000000 sims: 8.0013275
Total time: 37.73



### Analytical Solution

Is this just the expected value of the binomial distribution? I think that makes sense since we are basically looking at `expected value at each step * total steps`

$E[X] = np$

Where n = 1 and p = 0.5, so:

$E[X] = 0.5$ 

We expect 0.5 competitors per step, which is also: 

$ size * E[X] = 16 * 0.5 = 8$

### Numpy: 

Still not leveraging numpy in a great way here, but removing a lot of the cruft from the prior simulation. 

In [7]:
import numpy as np

def bridgeGame(size = 16):
    """Simulate a single bridge game in numpy"""
    arr = np.random.randint(2, size=size) 
    deaths = 0
    for i in range(size):
        if random.choice([0, 1]) != arr[i]:
            deaths += 1
    return size - deaths

In [8]:
sims = [1_000, 10_000, 100_000, 1_000_000, 2_000_000]
for sim in sims:
    start = time.time()
    tot_alive = 0
    for _ in range(sim):
        tot_alive += bridgeGame(size = 16)
    end = time.time()
    print(f"Average survivors for {sim} sims: {tot_alive / sim}")
    print(f"Total time: {end - start:.2f}\n")

Average survivors for 1000 sims: 7.97
Total time: 0.03

Average survivors for 10000 sims: 7.9897
Total time: 0.13

Average survivors for 100000 sims: 7.99269
Total time: 1.10

Average survivors for 1000000 sims: 8.002944
Total time: 10.83

Average survivors for 2000000 sims: 7.998742
Total time: 21.46

